# Overview of [smolagents](https://huggingface.co/docs/smolagents)

In [1]:
import os

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from phoenix.otel import register
from smolagents import LiteLLMModel, ToolCallingAgent

from travel_agent.qdrant import client as qdrant_client
from travel_agent.retrieval.embedding.embedding_generation import MODELS_PROMPTS
from travel_agent.retrieval.smolagents.prompts import RUS_PROMPT_TEMPLATES
from travel_agent.retrieval.smolagents.tool import GetExistingAvailableRubricsTool, TravelReviewQueryTool

if not os.getenv("QDRANT_URL"):
    print("Set QDRANT_URL and QDRANT_API_KEY env")
    exit(1)

print(f"Available embed models:\n- {'\n- '.join(MODELS_PROMPTS.keys())}")

Available embed models:
- cointegrated/rubert-tiny2
- DeepPavlov/rubert-base-cased-sentence
- ai-forever/sbert_large_nlu_ru
- ai-forever/sbert_large_mt_nlu_ru
- sentence-transformers/distiluse-base-multilingual-cased-v1
- sentence-transformers/distiluse-base-multilingual-cased-v2
- sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
- sentence-transformers/paraphrase-multilingual-mpnet-base-v2
- intfloat/multilingual-e5-large
- intfloat/multilingual-e5-base
- intfloat/multilingual-e5-small
- ai-forever/ru-en-RoSBERTa
- sergeyzh/BERTA


In [2]:
# OPTIONAL: add tracing
# First, run server in background: uv run -m phoenix.server.main serve
# Then, run this cell


tracer_provider = register(project_name="travel-agent-local")
SmolagentsInstrumentor().instrument(
    tracer_provider=tracer_provider
)  # automatically captures any smolagents calls as traces

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: travel-agent-local
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
# Replace with your favourite local model
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0"
model_id = f"ollama_chat/{MODEL_NAME}"


model = LiteLLMModel(
    model_id=model_id,
    api_base=os.getenv("OLLAMA_URL", "http://127.0.0.1:11434"),
    num_ctx=8192,  # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

test_response = model([{"role": "user", "content": [{"text": "Алло, это Кизару у телефона?", "type": "text"}]}]).content
print(test_response)
assert test_response

Здравствуйте! Да, это Кизару. (Учитывая, что я — виртуальный персонаж.) Как я могу вам помочь?


In [17]:
embed_model_name = "intfloat/multilingual-e5-base"
collection_name = "moskva_intfloat_multilingual_e5_base"


get_available_rubrics_tool = GetExistingAvailableRubricsTool()
review_search_tool = TravelReviewQueryTool(embed_model_name, qdrant_client, collection_name)


agent = ToolCallingAgent(
    tools=[review_search_tool, get_available_rubrics_tool], model=model, max_steps=10, verbosity_level=2
)

2025-04-26 19:44:30.024 | INFO     | travel_agent.retrieval.smolagents.tool:__init__:84 - Using device: mps


In [18]:
print(agent.system_prompt)

You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer"

In [19]:
# Sanity check
agent.run("Что ты думаешь о римской империи?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Что ты думаешь о римской империи?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
"name": "final_answer",                                                                                            
"arguments": {"answer": "Римская империя была выдающимся государственным образованием с богатым историческим       
наследием. По некоторым оценкам и отзывам историков, основные факты и достижения включают:\n- Расцвет в области    
права (правосудие, правовые нормы), которое до сих пор влияет на современное законодательство.\n- Эффективная      
система управления и инфраструктуры: дороги, мосты, акведуки, которые облегчали торговлю и распространение         
культуры.\n- Величественные архитектурные памятники (Колизей, Форум Римский) и искусство.\n- Сильную армию,        
обеспечивающую порядок в империи на протяжении столетий.\nОднако, следует отметить проблемы с коррупцией,          
социальное неравенство и военные конфликты, которые привели к её упадку."}                                         
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Римская империя была выдающимся государственным        │
│ образованием с богатым историческим наследием. По некоторым оценкам и отзывам историков, основные факты и       │
│ достижения включают:\n- Расцвет в области права (правосудие, правовые нормы), которое до сих пор влияет на      │
│ современное законодательство.\n- Эффективная система управления и инфраструктуры: дороги, мосты, акведуки,      │
│ которые облегчали торговлю и распространение культуры.\n- Величественные архитектурные памятники (Колизей,      │
│ Форум Римский) и искусство.\n- Сильную армию, обеспечивающую порядок в империи на протяжении столетий.\nОднако, │
│ следует отметить проблемы с коррупцией, социальное неравенство и военные конфликты, которые привели к её        │
│ упадку.'}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Римская империя была выдающимся государственным образованием с богатым историческим наследием. По 
некоторым оценкам и отзывам историков, основные факты и достижения включают:
- Расцвет в области права (правосудие, правовые нормы), которое до сих пор влияет на современное законодательство.
- Эффективная система управления и инфраструктуры: дороги, мосты, акведуки, которые облегчали торговлю и 
распространение культуры.
- Величественные архитектурные памятники (Колизей, Форум Римский) и искусство.
- Сильную армию, обеспечивающую порядок в империи на протяжении столетий.
Однако, следует отметить проблемы с коррупцией, социальное неравенство и военные конфликты, которые привели к её 
упадку.

[Step 1: Duration 18.16 seconds| Input tokens: 1,820 | Output tokens: 207]

'Римская империя была выдающимся государственным образованием с богатым историческим наследием. По некоторым оценкам и отзывам историков, основные факты и достижения включают:\n- Расцвет в области права (правосудие, правовые нормы), которое до сих пор влияет на современное законодательство.\n- Эффективная система управления и инфраструктуры: дороги, мосты, акведуки, которые облегчали торговлю и распространение культуры.\n- Величественные архитектурные памятники (Колизей, Форум Римский) и искусство.\n- Сильную армию, обеспечивающую порядок в империи на протяжении столетий.\nОднако, следует отметить проблемы с коррупцией, социальное неравенство и военные конфликты, которые привели к её упадку.'

In [20]:
print(agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{"name": "travel_review_query", "arguments": {"query": "посоветуй хороший японский ресторан в Москве",             
"min_rating": 4}}                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'посоветуй хороший японский ресторан в Москве',   │
│ 'min_rating': 4}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Izumi ===
Адрес: Москва, Мясницкая улица, 38с1
Рейтинг: 5.0
Категории: |'Ресторан', 'Бар, паб', 'Караоке-клуб']
Текст: Одно из самых любимых заведений японской кухни в Москве!  За адекватную цену вы получите действительно 
вкусное, похожее на настоящее, японское блюдо. Разнообразное меню и изобилие различных видов саке. Приятный 
интерьер. Отдельно хочется отметить удобную функцию вызова официанта с помощью кнопочки на столе :) 

=== Отзыв на Tsunami Sushi & Cocktails ===
Адрес: Москва, Благовещенский переулок, 1А
Рейтинг: 5.0
Категории: |'Ресторан', 'Бар, паб', 'Суши-бар']
Текст: Бомбически! Хотела поесть действительно вкусных роллов и не прогадала с выбором места. Приятный интерьер в 
стиле неонового Токио, техно/хаус музыка, которая не отвлекает от ужина, а только добавляет атмосферы. Посадка 
практически полная, но разговоры не мешали, шумно не было.  Брали роллы Мега Лосось, Радуга, Нигири с лососем и оши
с гребешком. Очень вкусно и очень сытно. Рыба свежая, сочетания вкусов потрясающие. Муж брал коктейли и тоже 
остался доволен.  UPD 17 июня 2023: Выбрала Tsunami, чтобы вдвоём с мужем отпраздновать мой день рождения. Как 
всегда безукаризненное обслуживание, наивкуснейшая еда, шикарная атмосфера. Пока люимое заведение японской кухни в 
Москве, куда действительно хочется возвращаться. 

=== Отзыв на Тануки ===
Адрес: Москва, Новокосинская улица, 36
Рейтинг: 4.0
Категории: |'Ресторан', 'Суши-бар', 'Доставка еды и обедов']
Текст: Всем привет! Идём в "Тануки"): роллы и суши да, вкусно, качественно, порции конечно уменьшились, но в целом 
приемлимо. С детьми там можно отдохнуть, есть детская комната, в сбб и вс работают аниматоры, шум и гам стоит 
знатный!)). Если идти туда  без детей  на любителя,  было бы другое приличное место, хоть маленький ресторанчик, в 
Новокосино, я бы выбрала другое, без детских воплей и криков. А так  добро пожаловать, выпейте побольше пива, или 
что покрепче и шум вам ни почём!)😂 Обслуживание, сервис  хорошие, но не сказать, что бы впечатляющие🤷‍♂️ Вобщем, в 
Новокосино у вас всего 2 варианта приличного кафе (на ресторан это не тянет, хоть как посмотри) либо японская кухня
и это "Тануки", либо узбекская и это "Чайхона",  удивительно, что в столице России приходится делать такой выбор! 
Если отзыв прочитает какойлибо ресторатор, то  прямо обращаюсь: "Откройте пожалуйста в Новокосино   кафе, с  
привычной нам всем русской кухней!" 

=== Отзыв на Izumi ===
Адрес: Москва, Мясницкая улица, 38с1
Рейтинг: 5.0
Категории: |'Ресторан', 'Бар, паб', 'Караоке-клуб']
Текст: Прекрасное место с великолепным тихим антуражем. Был днём, с друзьями. Еда от японского шефа на любой вкус, 
и для тех кто не любит рыбу и морепродукты и для тех кто их обожает. Я лично ел Тонкацу Рамен, мне очень 
понравилось, в отличии от большинства мест в Москве где всëтаки готовится под ожидания русских, тут именно что 
необычный вкус явно не из того  тех что можно попробовать в други местах.   Именно тут я пожалуй понял что именно 
японцы имеют ввиду по умами вкусом, реально очень плотный и при этом совершенно нежирный бульон просто пропитан 
вкусом который в других кухнях встречается довольно редко даже при самом поедании мяса. Однозначно рекомендую, 
однако предупреждаю что напитки как алко так и без состоят изо льда на 80%. 

=== Отзыв на Якитория ===
Адрес: Москва, улица Маршала Полубоярова, 1
Рейтинг: 5.0
Категории: |'Суши-бар', 'Ресторан', 'Доставка еды и обедов']
Текст: Очень хороший  ресторанчик японской кухни. Заказываю всегда отсюда если хочется роллов. Очень вкусные 
горячие роллы. Детям также будет интересно перекусить тут.

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
  "name": "final_answer",                                                                                          
  "arguments": {"answer": "На основе положительных отзывов рекомендую посетить ресторан **Izumi** по адресу Москва,
Мясницкая улица, 38с1. Отмечают разнообразие меню и высокое качество блюд японской кухни. Также стоит обратить     
внимание на Tsunami Sushi & Cocktails в Благовещенском переулке за атмосферу неонового Токио и отличные коктейли."}
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'На основе положительных отзывов рекомендую посетить    │
│ ресторан **Izumi** по адресу Москва, Мясницкая улица, 38с1. Отмечают разнообразие меню и высокое качество блюд  │
│ японской кухни. Также стоит обратить внимание на Tsunami Sushi & Cocktails в Благовещенском переулке за         │
│ атмосферу неонового Токио и отличные коктейли.'}                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: На основе положительных отзывов рекомендую посетить ресторан **Izumi** по адресу Москва, Мясницкая 
улица, 38с1. Отмечают разнообразие меню и высокое качество блюд японской кухни. Также стоит обратить внимание на 
Tsunami Sushi & Cocktails в Благовещенском переулке за атмосферу неонового Токио и отличные коктейли.

[Step 2: Duration 22.63 seconds| Input tokens: 5,123 | Output tokens: 161]

На основе положительных отзывов рекомендую посетить ресторан **Izumi** по адресу Москва, Мясницкая улица, 38с1. Отмечают разнообразие меню и высокое качество блюд японской кухни. Также стоит обратить внимание на Tsunami Sushi & Cocktails в Благовещенском переулке за атмосферу неонового Токио и отличные коктейли.


In [21]:
agent.memory.steps

[TaskStep(task='Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?', task_images=None),
 ActionStep(model_input_messages=[{'role': <MessageRole.SYSTEM: 'system'>, 'content': [{'type': 'text', 'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\

In [22]:
print(agent.run("Можешь ли посоветовать гостиницы в Москве?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Можешь ли посоветовать гостиницы в Москве?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
  "name": "travel_review_query",                                                                                   
  "arguments": {                                                                                                   
    "query": "Гостиницы Москва, рекомендации",                                                                     
    "min_rating": 4                                                                                                
  }                                                                                                                
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'Гостиницы Москва, рекомендации', 'min_rating':   │
│ 4}                                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Вега Измайлово Конгресс-центр ===
Адрес: Москва, Измайловское шоссе, 71, корп. 3В
Рейтинг: 5.0
Категории: |'Организация конференций и семинаров', 'Конференц-зал']
Текст: Моя "домашняя" гостиница в Москве, больше двадцати лет стараюсь останавливаться в ней. Застал ещё 
"олимпийский" дизайн и весь процесс модернизации наблюдал изнутри. Однозначно  рекомендую. 

=== Отзыв на Националь ===
Адрес: Москва, Моховая улица, 15/1с1
Рейтинг: 5.0
Категории: |'Гостиница']
Текст: Обслуживание на отлично, удобное расположение гостиницы, утные и чистые номера, вкусная еда, интересные 
завтраки, вежливый персонал. Располагает, чтоб вернуться снова. Одна из лучших гостиниц Москвы. Рекомендую. 

=== Отзыв на Хилтон Москва Ленинградская ===
Адрес: Москва, Центральный административный округ, Красносельский район, Каланчёвская улица, 21/40
Рейтинг: 5.0
Категории: |'Гостиница']
Текст: Хорошая гостиница в центре города Москвы.С гостиницы очень хороший и красивый вид.Персонал вежливый и 
приятный.Цена выше среднего.А так всем рекомендую)) 

=== Отзыв на Аэростар ===
Адрес: Москва, Ленинградский проспект, 37, корп. 9
Рейтинг: 5.0
Категории: |'Гостиница']
Текст: Гостиница хорошая. Номера, те которые отремонтированы, стильные просторные. Завтрак хороший, есть парковка. 
Расположение тоже норм, до метро не успешным шагом 57 минут. Для проведения выходных в Москве  самое то, 
рекомендую! 

=== Отзыв на СтандАрт ===
Адрес: Москва, Страстной бульвар, 2
Рейтинг: 5.0
Категории: |'Гостиница']
Текст: Последние три года приезжаю в Москву на переговоры и останавливаюсь в этой гостинице. Очень удобное место 
расположение отеля в центре столицы. После дня переговоров можно прогуляться по ночному городу, а Москва ночью 
просто великолепна! В самой гостинице тепло и уютно в номерах. Белье белоснежное, кровать большая , матрас удобный 
спится просто здорово. Завтрак разнообразный шведский стол плюс персонал, который помогает дополнительно не только 
выбрать блюда, но и приготовит отдельно блюда по вашему желанию. Отдельно хотел отметить работу персонала при 
заселении: прилетели в столицу уже ночью, заселение было поздним, тем не менее персонал отработал процедуру быстро 
в течении пары минут при этом решил наше пожелание по раннему завтраку. Одним словом у нас этот приезд прошел 
просто великолепно да и переговоры прошли результативно. Благодарность персоналу гостиницы за отличную организацию 
нашего пребывания в отеле. Хочется пожелать и дальше поддерживать уровень обслуживания на таком высоком уровне. С 
уважением  Владимир

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{"name": "final_answer", "arguments": {"answer": "На основе отзывов могу порекомендовать следующие гостиницы в     
Москве: 1) Вега Измайлово Конгресс-центр, 2) Националь, 3) Хилтон Москва Ленинградская, 4) Аэростар и 5) СтандАрт. 
Все они получили высокие оценки за качество обслуживания, удобное расположение и комфортные номера."}}             

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'На основе отзывов могу порекомендовать следующие       │
│ гостиницы в Москве: 1) Вега Измайлово Конгресс-центр, 2) Националь, 3) Хилтон Москва Ленинградская, 4) Аэростар │
│ и 5) СтандАрт. Все они получили высокие оценки за качество обслуживания, удобное расположение и комфортные      │
│ номера.'}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: На основе отзывов могу порекомендовать следующие гостиницы в Москве: 1) Вега Измайлово 
Конгресс-центр, 2) Националь, 3) Хилтон Москва Ленинградская, 4) Аэростар и 5) СтандАрт. Все они получили высокие 
оценки за качество обслуживания, удобное расположение и комфортные номера.

[Step 2: Duration 17.49 seconds| Input tokens: 4,648 | Output tokens: 152]

На основе отзывов могу порекомендовать следующие гостиницы в Москве: 1) Вега Измайлово Конгресс-центр, 2) Националь, 3) Хилтон Москва Ленинградская, 4) Аэростар и 5) СтандАрт. Все они получили высокие оценки за качество обслуживания, удобное расположение и комфортные номера.


In [23]:
# translated prompts

agent = ToolCallingAgent(
    tools=[review_search_tool],
    model=model,
    prompt_templates=RUS_PROMPT_TEMPLATES,
    max_steps=3,
    verbosity_level=2,  # DEBUG
)

print(agent.system_prompt)

Вы — эксперт-ассистент, который может решать любые задачи с использованием вызовов инструментов. Вам будет дана задача, которую вы должны решить наилучшим образом.
Для этого вам предоставлен доступ к некоторым инструментам.
Вызов инструмента, который вы пишете, является действием: после выполнения инструмента вы получите результат вызова инструмента как "наблюдение".
Эта последовательность Действие/Наблюдение может повторяться N раз, вам следует выполнять несколько шагов, когда это необходимо.
Вы можете использовать результат предыдущего действия как ввод для следующего действия.
Наблюдение всегда будет строкой: она может представлять файл, например "image_1.jpg".
Затем вы можете использовать его как ввод для следующего действия. Вы можете сделать это, например, следующим образом:

Наблюдение: "image_1.jpg"

Действие:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}
Чтобы предоставить окончательный ответ на задачу, используйте блок действия с "name": "final_an

In [24]:
resp = agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{"name": "travel_review_query", "arguments": {"query": "посоветуй хороший японский ресторан в Москве",             
"min_rating": 4}}                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'посоветуй хороший японский ресторан в Москве',   │
│ 'min_rating': 4}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Izumi ===
Адрес: Москва, Мясницкая улица, 38с1
Рейтинг: 5.0
Категории: |'Ресторан', 'Бар, паб', 'Караоке-клуб']
Текст: Одно из самых любимых заведений японской кухни в Москве!  За адекватную цену вы получите действительно 
вкусное, похожее на настоящее, японское блюдо. Разнообразное меню и изобилие различных видов саке. Приятный 
интерьер. Отдельно хочется отметить удобную функцию вызова официанта с помощью кнопочки на столе :) 

=== Отзыв на Tsunami Sushi & Cocktails ===
Адрес: Москва, Благовещенский переулок, 1А
Рейтинг: 5.0
Категории: |'Ресторан', 'Бар, паб', 'Суши-бар']
Текст: Бомбически! Хотела поесть действительно вкусных роллов и не прогадала с выбором места. Приятный интерьер в 
стиле неонового Токио, техно/хаус музыка, которая не отвлекает от ужина, а только добавляет атмосферы. Посадка 
практически полная, но разговоры не мешали, шумно не было.  Брали роллы Мега Лосось, Радуга, Нигири с лососем и оши
с гребешком. Очень вкусно и очень сытно. Рыба свежая, сочетания вкусов потрясающие. Муж брал коктейли и тоже 
остался доволен.  UPD 17 июня 2023: Выбрала Tsunami, чтобы вдвоём с мужем отпраздновать мой день рождения. Как 
всегда безукаризненное обслуживание, наивкуснейшая еда, шикарная атмосфера. Пока люимое заведение японской кухни в 
Москве, куда действительно хочется возвращаться. 

=== Отзыв на Тануки ===
Адрес: Москва, Новокосинская улица, 36
Рейтинг: 4.0
Категории: |'Ресторан', 'Суши-бар', 'Доставка еды и обедов']
Текст: Всем привет! Идём в "Тануки"): роллы и суши да, вкусно, качественно, порции конечно уменьшились, но в целом 
приемлимо. С детьми там можно отдохнуть, есть детская комната, в сбб и вс работают аниматоры, шум и гам стоит 
знатный!)). Если идти туда  без детей  на любителя,  было бы другое приличное место, хоть маленький ресторанчик, в 
Новокосино, я бы выбрала другое, без детских воплей и криков. А так  добро пожаловать, выпейте побольше пива, или 
что покрепче и шум вам ни почём!)😂 Обслуживание, сервис  хорошие, но не сказать, что бы впечатляющие🤷‍♂️ Вобщем, в 
Новокосино у вас всего 2 варианта приличного кафе (на ресторан это не тянет, хоть как посмотри) либо японская кухня
и это "Тануки", либо узбекская и это "Чайхона",  удивительно, что в столице России приходится делать такой выбор! 
Если отзыв прочитает какойлибо ресторатор, то  прямо обращаюсь: "Откройте пожалуйста в Новокосино   кафе, с  
привычной нам всем русской кухней!" 

=== Отзыв на Izumi ===
Адрес: Москва, Мясницкая улица, 38с1
Рейтинг: 5.0
Категории: |'Ресторан', 'Бар, паб', 'Караоке-клуб']
Текст: Прекрасное место с великолепным тихим антуражем. Был днём, с друзьями. Еда от японского шефа на любой вкус, 
и для тех кто не любит рыбу и морепродукты и для тех кто их обожает. Я лично ел Тонкацу Рамен, мне очень 
понравилось, в отличии от большинства мест в Москве где всëтаки готовится под ожидания русских, тут именно что 
необычный вкус явно не из того  тех что можно попробовать в други местах.   Именно тут я пожалуй понял что именно 
японцы имеют ввиду по умами вкусом, реально очень плотный и при этом совершенно нежирный бульон просто пропитан 
вкусом который в других кухнях встречается довольно редко даже при самом поедании мяса. Однозначно рекомендую, 
однако предупреждаю что напитки как алко так и без состоят изо льда на 80%. 

=== Отзыв на Якитория ===
Адрес: Москва, улица Маршала Полубоярова, 1
Рейтинг: 5.0
Категории: |'Суши-бар', 'Ресторан', 'Доставка еды и обедов']
Текст: Очень хороший  ресторанчик японской кухни. Заказываю всегда отсюда если хочется роллов. Очень вкусные 
горячие роллы. Детям также будет интересно перекусить тут.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{ "name": "final_answer", "arguments": { "answer": "В Москве рекомендуем посетить следующие японские               
рестораны:\n\n- **Izumi** на Мясницкой улице, 38с1 – выделяется качеством блюд и разнообразным меню.\n- **Tsunami  
Sushi & Cocktails** в Благовещенском переулке, 1А – отличный выбор для любителей атмосферных заведений с неоновым  
Токио стилем и вкусными роллами.\n\nОба этих места получили высокие оценки посетителей за качество еды и           
обслуживания." } }                                                                                                 

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'В Москве рекомендуем посетить следующие японские       │
│ рестораны:\n\n- **Izumi** на Мясницкой улице, 38с1 – выделяется качеством блюд и разнообразным меню.\n-         │
│ **Tsunami Sushi & Cocktails** в Благовещенском переулке, 1А – отличный выбор для любителей атмосферных          │
│ заведений с неоновым Токио стилем и вкусными роллами.\n\nОба этих места получили высокие оценки посетителей за  │
│ качество еды и обслуживания.'}                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: В Москве рекомендуем посетить следующие японские рестораны:

- **Izumi** на Мясницкой улице, 38с1 – выделяется качеством блюд и разнообразным меню.
- **Tsunami Sushi & Cocktails** в Благовещенском переулке, 1А – отличный выбор для любителей атмосферных заведений 
с неоновым Токио стилем и вкусными роллами.

Оба этих места получили высокие оценки посетителей за качество еды и обслуживания.

[Step 2: Duration 25.23 seconds| Input tokens: 4,806 | Output tokens: 192]

In [25]:
resp = agent.run("Подскажи мне ателье по ременту обуви?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Подскажи мне ателье по ременту обуви?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{"name": "travel_review_query", "arguments": {"query": "отличное ателье по ремонту обуви"}}                        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'отличное ателье по ремонту обуви'}               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Люкс мастер ===
Адрес: Москва, Кожевническая улица, 10, стр. 1
Рейтинг: 5.0
Категории: |'Ремонт обуви', 'Металлоремонт', 'Ремонт одежды']
Текст: Пользуюсь услугами ремонта обуви  и ателье уже почти год. Всем довольна. Качество работы отличное. Цены 
приемлемые. Если нужно быстро  сделают. А самое главное ребята приятные в общении и учитывают все пожелания. 
Рекомендую!) 

=== Отзыв на Дом Быта ===
Адрес: Москва, Зарайская улица, 53
Рейтинг: 5.0
Категории: |'Ремонт одежды', 'Ремонт обуви', 'Изготовление и ремонт ключей']
Текст: Отличное Ателье! Пожалуй , самое лучшее соотношение качества и цены в нашем районе!!! Исполняют все заказы в
срок и качественно. Персонал очень вежливый. Ремонтировала пальто и брюки. Все понравилось!  Рекомендую 🏆🥇👍🏻 

=== Отзыв на Ателье ===
Адрес: Москва, проезд Дежнёва, 21
Рейтинг: 5.0
Категории: |'Ремонт одежды', 'Ремонт обуви', 'Ателье по пошиву одежды']
Текст: Постоянно пользуюсь услугами данного ателье! Все на высшем уровне.От ремонта обуви до ремонта кожаных и 
меховых изделий. Спасибо вам за ваш труд 

=== Отзыв на Косино Парк ===
Адрес: Москва, Святоозёрская улица, 1А
Рейтинг: 5.0
Категории: |'Торговый центр', 'Развлекательный центр']
Текст: Дом быта. Ремонт обуви делают очень быстро и качественно. Мастер раньше работал на Рязанском проспекте. 
Теперь в этом ТЦ. Зовут Гена. Очень вежливый и работу свою знает. Поэтому езжу к нему. Вся обувь идеальна после его
ремонта.  Рекомендую.  

=== Отзыв на Afanas’ev ===
Адрес: Москва, Малый Лёвшинский переулок, 6-8
Рейтинг: 5.0
Категории: |'Ремонт обуви', 'Кожевенная мастерская', 'Химчистка']
Текст: Большое спасибо за вашу работу !! Высркий профессианолизм и безупречное качество . Вежливое обслуживание. Вы
сломали мое стереотипное представление об ателье по ремонту обуви!)

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{ "name": "final_answer", "arguments": {"answer": "Наиболее рекомендуемые места для ремонта обуви в Москве:\n\nЛюкс
мастер - Москва, Кожевническая улица, 10, стр. 1 \nДом Быта - Москва, Зарайская улица, 53 (также есть филиал в     
Косино Парк)\n\nОба этих ателье имеют отличные отзывы и высокий рейтинг по ремонту обуви."}}                       

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Наиболее рекомендуемые места для ремонта обуви в       │
│ Москве:\n\nЛюкс мастер - Москва, Кожевническая улица, 10, стр. 1 \nДом Быта - Москва, Зарайская улица, 53       │
│ (также есть филиал в Косино Парк)\n\nОба этих ателье имеют отличные отзывы и высокий рейтинг по ремонту         │
│ обуви.'}                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Наиболее рекомендуемые места для ремонта обуви в Москве:

Люкс мастер - Москва, Кожевническая улица, 10, стр. 1 
Дом Быта - Москва, Зарайская улица, 53 (также есть филиал в Косино Парк)

Оба этих ателье имеют отличные отзывы и высокий рейтинг по ремонту обуви.

[Step 2: Duration 16.47 seconds| Input tokens: 4,663 | Output tokens: 155]